In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

sns.set_context("paper")
import matplotlib.ticker as ticker

from tqdm import tqdm # progress bars :)
tqdm.pandas()

In [3]:
ep = pd.read_csv("gtdb-rs202.evolpaths.merged.csv.gz")
ep

,comparison_name,anchor_name,compare_name,path,lowest_common_rank,alphabet,ksize,scaled,jaccard,max_containment,...,genes_in_anchor,genes_in_ref,orthologous_genes,orthologous_fraction,pyani_ident,pyani_coverage,pyani_aln_length,pyani_sim_errors,pyani_hadamard,min_containment_ani
0,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,21,1,0.155515,0.401210,...,1804.0,3387.0,1353.0,75.0,0.947258,0.827762,1520244.0,80181.0,0.784104,0.926776
1,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,21,1000,0.160974,0.412247,...,1804.0,3387.0,1353.0,75.0,0.947258,0.827762,1520244.0,80181.0,0.784104,0.928150
2,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,21,2000,0.159965,0.417136,...,1804.0,3387.0,1353.0,75.0,0.947258,0.827762,1520244.0,80181.0,0.784104,0.927531
3,GCA_004524385.1_x_GCA_011364975.1,GCA_004524385.1,GCA_011364975.1,path1,family,nucleotide,21,1,0.000272,0.000828,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.689328
4,GCA_004524385.1_x_GCA_011364975.1,GCA_004524385.1,GCA_011364975.1,path1,family,nucleotide,21,1000,0.000712,0.002187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.721482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589657,GCA_002409405.1_x_GCA_003567135.1,GCA_002409405.1,GCA_003567135.1,path999,phylum,protein,11,100,0.001703,0.003699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.592279
589658,GCA_002409405.1_x_GCA_003567135.1,GCA_002409405.1,GCA_003567135.1,path999,phylum,protein,11,200,0.001819,0.003919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.596179
589659,GCA_002409405.1_x_GCF_004319585.1,GCA_002409405.1,GCF_004319585.1,path999,superkingdom,protein,11,1,0.000687,0.001424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.547567
589660,GCA_002409405.1_x_GCF_004319585.1,GCA_002409405.1,GCF_004319585.1,path999,superkingdom,protein,11,100,0.000427,0.000885,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.524458


In [4]:
ep.columns

Index(['comparison_name', 'anchor_name', 'compare_name', 'path',
       'lowest_common_rank', 'alphabet', 'ksize', 'scaled', 'jaccard',
       'max_containment', 'anchor_containment', 'compare_containment',
       'anchor_hashes', 'compare_hashes', 'num_common', 'jaccard_ani',
       'jani_low', 'jani_high', 'containA_ani', 'containAani_low',
       'containAani_high', 'containBani', 'containBani_low',
       'containBani_high', 'mcANI', 'mcANI_low', 'mcANI_high', 'alpha-ksize',
       'compareM_mean_aai', 'compareM_std_aai', 'genes_in_anchor',
       'genes_in_ref', 'orthologous_genes', 'orthologous_fraction',
       'pyani_ident', 'pyani_coverage', 'pyani_aln_length', 'pyani_sim_errors',
       'pyani_hadamard', 'min_containment_ani'],
      dtype='object')

In [5]:
prot_only = ep[ep["alphabet"] == "protein"]
prot_only = prot_only[prot_only["pyani_ident"].notna()] # only grab comparisons we can work with
prot_only

,comparison_name,anchor_name,compare_name,path,lowest_common_rank,alphabet,ksize,scaled,jaccard,max_containment,...,genes_in_anchor,genes_in_ref,orthologous_genes,orthologous_fraction,pyani_ident,pyani_coverage,pyani_aln_length,pyani_sim_errors,pyani_hadamard,min_containment_ani
54,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,protein,7,1,0.253707,0.606440,...,1804.0,3387.0,1353.0,75.00,0.947258,0.827762,1520244.0,80181.0,0.784104,0.843463
55,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,protein,7,100,0.257015,0.615836,...,1804.0,3387.0,1353.0,75.00,0.947258,0.827762,1520244.0,80181.0,0.784104,0.844403
56,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,protein,7,200,0.256108,0.608974,...,1804.0,3387.0,1353.0,75.00,0.947258,0.827762,1520244.0,80181.0,0.784104,0.844570
72,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,protein,8,1,0.239671,0.580961,...,1804.0,3387.0,1353.0,75.00,0.947258,0.827762,1520244.0,80181.0,0.784104,0.856555
73,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,protein,8,100,0.230212,0.568399,...,1804.0,3387.0,1353.0,75.00,0.947258,0.827762,1520244.0,80181.0,0.784104,0.852504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589627,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,protein,10,100,0.328203,0.513314,...,3085.0,3163.0,2382.0,77.21,0.867239,0.764598,2645579.0,351229.0,0.663089,0.928546
589628,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,protein,10,200,0.326541,0.508864,...,3085.0,3163.0,2382.0,77.21,0.867239,0.764598,2645579.0,351229.0,0.663089,0.928614
589644,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,protein,11,1,0.308882,0.488857,...,3085.0,3163.0,2382.0,77.21,0.867239,0.764598,2645579.0,351229.0,0.663089,0.931143
589645,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,protein,11,100,0.311802,0.491692,...,3085.0,3163.0,2382.0,77.21,0.867239,0.764598,2645579.0,351229.0,0.663089,0.931862


In [10]:
prot_only[["compareM_mean_aai", "compareM_std_aai", "jani_low", "jani_high"]]

,compareM_mean_aai,compareM_std_aai,jani_low,jani_high
54,0.9242,0.0952,0.878125,0.879436
55,0.9242,0.0952,0.876950,0.883182
56,0.9242,0.0952,0.875328,0.884076
72,0.9242,0.0952,0.887359,0.888658
73,0.9242,0.0952,0.881371,0.887403
...,...,...,...,...
589627,0.9137,0.0849,0.930341,0.933536
589628,0.9137,0.0849,0.929338,0.933812
589644,0.9137,0.0849,0.933632,0.934408
589645,0.9137,0.0849,0.933093,0.936154


In [6]:
def aai_in_ci(row):
    row["aai_in_mc_ci"] = False
    row["aai_in_jaccard_ci"] = False
    compareM_mean = float(row["compareM_mean_aai"])
    compareM_stdev = float(row["compareM_std_aai"])
    compareM_low = compareM_mean - compareM_stdev
    compareM_high = compareM_mean + compareM_stdev
    mc_low = float(row["mcANI_low"])
    mc_high = float(row["mcANI_high"])
    if (mc_low <= compareM_low) & (compareM_high <= mc_high):
        row["aai_in_mc_ci"] = True
    jaai_low = float(row["jani_low"])
    jaai_high = float(row["jani_high"])
    if (jaai_low <= compareM_low) & (compareM_high <= jaai_high):
        row["aai_in_jaccard_ci"] = True
    return row

In [8]:
prot_only = prot_only.progress_apply(aai_in_ci, axis=1)
prot_only

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20160/20160 [00:45<00:00, 446.08it/s]


,comparison_name,anchor_name,compare_name,path,lowest_common_rank,alphabet,ksize,scaled,jaccard,max_containment,...,orthologous_genes,orthologous_fraction,pyani_ident,pyani_coverage,pyani_aln_length,pyani_sim_errors,pyani_hadamard,min_containment_ani,aai_in_mc_ci,aai_in_jaccard_ci
54,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,protein,7,1,0.253707,0.606440,...,1353.0,75.00,0.947258,0.827762,1520244.0,80181.0,0.784104,0.843463,False,False
55,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,protein,7,100,0.257015,0.615836,...,1353.0,75.00,0.947258,0.827762,1520244.0,80181.0,0.784104,0.844403,False,False
56,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,protein,7,200,0.256108,0.608974,...,1353.0,75.00,0.947258,0.827762,1520244.0,80181.0,0.784104,0.844570,False,False
72,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,protein,8,1,0.239671,0.580961,...,1353.0,75.00,0.947258,0.827762,1520244.0,80181.0,0.784104,0.856555,False,False
73,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,protein,8,100,0.230212,0.568399,...,1353.0,75.00,0.947258,0.827762,1520244.0,80181.0,0.784104,0.852504,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589627,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,protein,10,100,0.328203,0.513314,...,2382.0,77.21,0.867239,0.764598,2645579.0,351229.0,0.663089,0.928546,False,False
589628,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,protein,10,200,0.326541,0.508864,...,2382.0,77.21,0.867239,0.764598,2645579.0,351229.0,0.663089,0.928614,False,False
589644,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,protein,11,1,0.308882,0.488857,...,2382.0,77.21,0.867239,0.764598,2645579.0,351229.0,0.663089,0.931143,False,False
589645,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,protein,11,100,0.311802,0.491692,...,2382.0,77.21,0.867239,0.764598,2645579.0,351229.0,0.663089,0.931862,False,False


In [9]:
in_mcCI = prot_only[prot_only["aai_in_mc_ci"] ==True]
in_mcCI

,comparison_name,anchor_name,compare_name,path,lowest_common_rank,alphabet,ksize,scaled,jaccard,max_containment,...,orthologous_genes,orthologous_fraction,pyani_ident,pyani_coverage,pyani_aln_length,pyani_sim_errors,pyani_hadamard,min_containment_ani,aai_in_mc_ci,aai_in_jaccard_ci


In [12]:
in_mcCI.groupby(["scaled", "ksize"])[["aai_in_mc_ci"]].describe()

scaled


In [11]:
in_jCI = prot_only[prot_only["aai_in_jaccard_ci"] ==True]
in_jCI

,comparison_name,anchor_name,compare_name,path,lowest_common_rank,alphabet,ksize,scaled,jaccard,max_containment,...,orthologous_genes,orthologous_fraction,pyani_ident,pyani_coverage,pyani_aln_length,pyani_sim_errors,pyani_hadamard,min_containment_ani,aai_in_mc_ci,aai_in_jaccard_ci


In [13]:
in_jCI.groupby(["scaled", "ksize"])[["aai_in_jaccard_ci"]].describe()

scaled
